In [ ]:
import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lsc
import torch
import h5py

from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
from deeplsd.geometry.viz_2d import plot_images, plot_lines

# Model config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = {
    'detect_lines': True,  # Whether to detect lines or only DF/AF
    'line_detection_params': {
        'merge': True,  # Whether to merge close-by lines
        'filtering': False,  # Whether to filter out lines based on the DF/AF. Use 'strict' to get an even stricter filtering
        'grad_thresh': 3,
        'grad_nfa': True,  # If True, use the image gradient and the NFA score of LSD to further threshold lines. We recommand using it for easy images, but to turn it off for challenging images (e.g. night, foggy, blurry images)
    }
}

# Load the model
ckpt = 'deeplsd_wireframe.tar'
ckpt = torch.load(str(ckpt), map_location='cpu')
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()


image_dir = '/mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images'
output_dir = '/mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out'  
edges_dir = '/mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out_txt' 

# Create the output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(edges_dir, exist_ok=True)

image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    # Load an image
    img = cv2.imread(os.path.join(image_dir, image_file))[:, :, ::-1]
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Detect (and optionally refine) the lines
    inputs = {'image': torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.}
    with torch.no_grad():
        out = net(inputs)
        pred_lines = out['lines'][0]

    # Plot the predictions
    plt.figure(figsize=(10, 5))
    plot_images([img], ['DeepLSD lines'], cmaps='gray')
    plot_lines([pred_lines], indices=range(1))
    plt.title(image_file)

    # Save the output image
    output_file_path = os.path.join(output_dir, f'output_{image_file}')
    plt.savefig(output_file_path, bbox_inches='tight')
    plt.close()  # Close the figure to free memory

    # Save edges data to a text file
    edges_file_path = os.path.join(edges_dir, f'edges_{image_file}.txt')
    with open(edges_file_path, 'w') as f:
        for line in pred_lines:
            f.write(f"{line.tolist()}\n")  # Save each line as a list

    print(f"Saved predictions for {image_file} to {output_file_path}")
    print(f"Saved edges data to {edges_file_path}")
    print(f"Predictions shape: {pred_lines.shape}")
    print(f"Image shape: {img.shape}")
    print(image_files)


Saved predictions for 20151101_072507.jpg to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out/output_20151101_072507.jpg
Saved edges data to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out_txt/edges_20151101_072507.jpg.txt
Predictions shape: (21, 2, 2)
Image shape: (480, 640, 3)
['20151101_072507.jpg', 'example.jpg', 'istockphoto-1349030917-612x612.jpg']
Saved predictions for example.jpg to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out/output_example.jpg
Saved edges data to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out_txt/edges_example.jpg.txt
Predictions shape: (458, 2, 2)
Image shape: (480, 640, 3)
['20151101_072507.jpg', 'example.jpg', 'istockphoto-1349030917-612x612.jpg']
Saved predictions for istockphoto-1349030917-612x612.jpg to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out/output_istockphoto-1349030917-612x612.jpg
Saved edges data to /mnt/c/Users/USER/Desktop/DeepLSD/DeepLSD/notebooks/images_out_txt

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>